<strong><font size="5">Face Recognition using One shot Learning</font></strong><br><br>

In [1]:
import mtcnn
import tensorflow as tf
import tensorflow.python.keras as keras
from mtcnn.mtcnn import MTCNN
from PIL import Image
import os
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from scipy.spatial.distance import cosine
from keras_vggface.vggface import VGGFace
from tensorflow.keras import  Model
from tensorflow.keras.layers import Input
from tensorflow.keras.applications.vgg16 import VGG16
import tensorflow.keras.backend as K

Using TensorFlow backend.
F:\Anaconda\envs\keras\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
F:\Anaconda\envs\keras\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
F:\Anaconda\envs\keras\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
F:\Anaconda\envs\keras\lib\site-packages\tensorflow\python\framework\dtypes.py:519: Fu

In [57]:
print(keras.__version__)
print(tf.__version__)

2.2.4-tf
1.14.0


<strong><font size="4">Model used: VGG16</font></strong>

In [3]:
import warnings; warnings.simplefilter('ignore')
vgg_face=keras.models.load_model('recognition.h5')
vgg_face.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_26 (ZeroPaddi (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_27 (ZeroPaddi (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_28 (ZeroPaddi (None, 114, 114, 64)      0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 112, 112, 128)    

<strong><font size='4'>Function Descriptions</font></strong><br><br><br><br>

<font size='4'>get_face</font><br><br>

Parameters:filename(str)<br>
Used to get co-ordinates of all the faces in an image provided by the parameter filename<br><br><br>

<font size='4'>preprocess</font><br><br>

Parameters:imagelist(list)<br>
Used to get preprocess all the images to make the compatible for training<br><br><br>


<font size='4'>similarity</font><br><br>

Parameters:yhatrue(tensor),yhatest(tensor)<br>
Used to compute cosine similarity between two vectors.<br><br><br>

<font size='4'>class_draw</font><br><br>

Parameters:train(str),test(str),result(list),rectangles(list)<br>
Used to plot the resulting bounding boxes around the images.<br><br><br>


<font size='4'>class_draw</font><br><br>

Parameters:trainpath(str),testpath(str)<br>
Heart of the code,used to for the recognition task.<br><br><br>



In [81]:
def get_face(filename):
    pixels=np.asarray(filename)
    detector=MTCNN()
    faces=detector.detect_faces(pixels)
    facecut=[]
    rectangles=[]
    for face in faces:
        x1,y1,width,height=face['box']
        x2,y2=x1+width,y1+height
        facecut.append(pixels[y1:y2,x1:x2])
        rectangles.append([x1,y1,width,height])
    return (facecut,rectangles)

def preprocess(imagelist):
    processedimage=[]
    for img in imagelist:
        img=Image.fromarray(img)
        img=tf.image.resize(img,(224,224))
        img=tf.reshape(img,shape=(1, img.shape[0], img.shape[1], img.shape[2]))
        img=keras.applications.vgg16.preprocess_input(img)
        processedimage.append(img)
    return processedimage

def similarity(yhatrue,yhatest):
    score=cosine(yhatrue,yhatest)
    return score

def class_draw(train,test,result,rectangles):
    ind=result.index(min(result))
    train_arr=plt.imread(train)
    test_arr=plt.imread(test)
    fig=plt.figure(figsize=(15,10))
    fig.add_subplot(1,2,1)
    plt.imshow(train_arr)
    plt.title('Train Image')
    plt.axis(False)
    
    fig.add_subplot(1,2,2)
    plt.imshow(test_arr)
    plt.title('Target Image')
    plt.axis(False)
    x1,y1,width,height=rectangles[ind]
    ax=plt.gca()
    rect=plt.Rectangle((x1,y1),width,height,fill=False,color='red',linewidth=2.0)
    ax.add_patch(rect)
    plt.show()
    
def classifier(trainpath,testpath):
    trainimg=keras.preprocessing.image.load_img(trainpath,[224,224])
    testimg=keras.preprocessing.image.load_img(testpath,[224,224])
    trainface=get_face(trainimg)[0]
    testfaces,rectangles=get_face(testimg)
    processed_train=preprocess(trainface)
    processed_test=preprocess(testfaces)
    #model=keras.applications.vgg16.VGG16(include_top=True,weights="imagenet")
    #model=keras.Model(inputs=model.inputs,outputs=model.layers[-2].output)
    #for layer in model.layers:
        #layer.trainable=False
    yhatrue=vgg_face.predict(processed_train[0],steps=1)
    yhatest=[]
    for _ in processed_test:
        yhatest.append(vgg_face.predict(_,steps=1))
    all_faces=[]
    for scores in yhatest:
        all_faces.append(similarity(yhatrue,scores))
    class_draw(trainpath,testpath,all_faces,rectangles)
